In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import sys
from sklearn.feature_selection import RFE
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('DatosSingapore.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2459 entries, 0 to 2458
Columns: 130 entries, id to room_type:Shared room
dtypes: bool(98), float64(20), int64(12)
memory usage: 850.2 KB


In [ ]:
pd.set_option('display.max_columns', None)
data.head()

,id,host_id,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_365,number_of_reviews,availability_eoy,estimated_occupancy_l365d,review_scores_rating,review_scores_communication,review_scores_location,instant_bookable,calculated_host_listings_count,amenities_number,number_of_private_bathrooms,number_of_shared_bathrooms,estimated_response_time_hours,host_response_time:a few days or more,host_response_time:within a day,host_response_time:within a few hours,host_response_time:within an hour,neighbourhood_cleansed:Ang Mo Kio,neighbourhood_cleansed:Bedok,neighbourhood_cleansed:Bishan,neighbourhood_cleansed:Bukit Batok,neighbourhood_cleansed:Bukit Merah,neighbourhood_cleansed:Bukit Panjang,neighbourhood_cleansed:Bukit Timah,neighbourhood_cleansed:Central Water Catchment,neighbourhood_cleansed:Changi,neighbourhood_cleansed:Choa Chu Kang,neighbourhood_cleansed:Clementi,neighbourhood_cleansed:Downtown Core,neighbourhood_cleansed:Geylang,neighbourhood_cleansed:Hougang,neighbourhood_cleansed:Jurong East,neighbourhood_cleansed:Jurong West,neighbourhood_cleansed:Kallang,neighbourhood_cleansed:Marina South,neighbourhood_cleansed:Marine Parade,neighbourhood_cleansed:Museum,neighbourhood_cleansed:Newton,neighbourhood_cleansed:Novena,neighbourhood_cleansed:Orchard,neighbourhood_cleansed:Outram,neighbourhood_cleansed:Pasir Ris,neighbourhood_cleansed:Punggol,neighbourhood_cleansed:Queenstown,neighbourhood_cleansed:River Valley,neighbourhood_cleansed:Rochor,neighbourhood_cleansed:Sembawang,neighbourhood_cleansed:Sengkang,neighbourhood_cleansed:Serangoon,neighbourhood_cleansed:Singapore River,neighbourhood_cleansed:Southern Islands,neighbourhood_cleansed:Tampines,neighbourhood_cleansed:Tanglin,neighbourhood_cleansed:Toa Payoh,neighbourhood_cleansed:Western Water Catchment,neighbourhood_cleansed:Woodlands,neighbourhood_cleansed:Yishun,neighbourhood_group_cleansed:Central Region,neighbourhood_group_cleansed:East Region,neighbourhood_group_cleansed:North Region,neighbourhood_group_cleansed:North-East Region,neighbourhood_group_cleansed:West Region,property_type:Boat,property_type:Entire condo,property_type:Entire guest suite,property_type:Entire guesthouse,property_type:Entire home,property_type:Entire loft,property_type:Entire rental unit,property_type:Entire serviced apartment,property_type:Entire townhouse,property_type:Private room,property_type:Private room in barn,property_type:Private room in bed and breakfast,property_type:Private room in bungalow,property_type:Private room in casa particular,property_type:Private room in castle,property_type:Private room in condo,property_type:Private room in earthen home,property_type:Private room in guest suite,property_type:Private room in guesthouse,property_type:Private room in home,property_type:Private room in hostel,property_type:Private room in loft,property_type:Private room in rental unit,property_type:Private room in serviced apartment,property_type:Private room in townhouse,property_type:Private room in train,property_type:Private room in vacation home,property_type:Private room in villa,property_type:Room in aparthotel,property_type:Room in boutique hotel,property_type:Room in hostel,property_type:Room in hotel,property_type:Room in serviced apartment,property_type:Shared room in bed and breakfast,property_type:Shared room in boutique hotel,property_type:Shared room in dome,property_type:Shared room in guesthouse,property_type:Shared room in hostel,property_type:Shared room in hotel,property_type:Shared room in rental unit,property_type:Tiny home,room_type:Entire home/apt,room_type:Hotel room,room_type:Private room,room_type:Shared room
0,71896,367042,1.0,1.0,False,10,True,True,1.3

In [ ]:
cols = ["review_scores_rating", "price", "amenities_number"]


In [ ]:
scaler = StandardScaler()
scaled = scaler.fit_transform(data[cols])

In [ ]:
data["simple_score"] = (
    scaled[:,0]        # rating
    - scaled[:,1]      # precio invertido
    + scaled[:,2]      # amenities
)

In [ ]:
threshold = data["simple_score"].median()
data["recommended"] = (data["simple_score"] >= threshold).astype(int)

In [ ]:
print("Proportions:")
print(data["recommended"].value_counts(normalize=True))

Proportions:
recommended
1    0.500203
0    0.499797
Name: proportion, dtype: float64


In [ ]:
data[["review_scores_rating","price","amenities_number","simple_score","recommended"]].head()

,review_scores_rating,price,amenities_number,simple_score,recommended
0,4.16,104.0,37,1.258388,1
1,4.41,76.0,25,0.914766,1
2,4.43,85.0,23,0.731859,1
3,4.60,41.0,45,3.157502,1
4,4.43,101.0,23,0.633637,1


In [ ]:
features = ["price","accommodates","bedrooms","bathrooms","amenities_number","review_scores_rating"]

X = data[features]
y = data['recommended']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.80, test_size = 0.20 ,
                                                   random_state = 42, stratify = y)


In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train_scaled, y_train)

LogisticRegression()

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
model.predict_proba(X_train_scaled)

array([[2.78441349e-04, 9.99721559e-01],
       [9.99922987e-01, 7.70126577e-05],
       [1.89098925e-01, 8.10901075e-01],
       [3.06170518e-02, 9.69382948e-01],
       [6.61405617e-07, 9.99999339e-01],
       [9.26155073e-01, 7.38449270e-02],
       [1.29151145e-08, 9.99999987e-01],
       [1.36484683e-03, 9.98635153e-01],
       [1.00000000e+00, 1.01475896e-10],
       [2.81228611e-01, 7.18771389e-01],
       [8.06454550e-01, 1.93545450e-01],
       [9.99999321e-01, 6.79200786e-07],
       [9.90577258e-01, 9.42274224e-03],
       [9.98952474e-01, 1.04752591e-03],
       [9.99896931e-01, 1.03068942e-04],
       [3.10603160e-01, 6.89396840e-01],
       [9.91721129e-01, 8.27887057e-03],
       [9.42739825e-01, 5.72601752e-02],
       [7.91673530e-01, 2.08326470e-01],
       [5.40459788e-07, 9.99999460e-01],
       [5.71407398e-05, 9.99942859e-01],
       [9.99985300e-01, 1.47000292e-05],
       [4.08893731e-05, 9.99959111e-01],
       [1.00000000e+00, 4.50837677e-10],
       [9.973476

In [ ]:
model.score(X_train_scaled, y_train)

0.9959328927300457

In [ ]:
model.coef_

array([[-6.18039418, -0.07799349, -0.28144887, -0.01989109,  6.3160247 ,
         6.26434676]])

In [ ]:
y_train_predi = model.predict(X_train_scaled)

In [ ]:
print(classification_report(y_train, y_train_predi))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       983
           1       0.99      1.00      1.00       984

    accuracy                           1.00      1967
   macro avg       1.00      1.00      1.00      1967
weighted avg       1.00      1.00      1.00      1967



In [ ]:
cf = confusion_matrix(y_train, y_train_predi)
print(cf)

[[978   5]
 [  3 981]]


In [ ]:
y_pred_test = model.predict(X_test_scaled)

In [ ]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98       264
           1       0.97      1.00      0.98       228

    accuracy                           0.98       492
   macro avg       0.98      0.98      0.98       492
weighted avg       0.98      0.98      0.98       492



In [ ]:
cft = confusion_matrix(y_test, y_pred_test)
print(cft)

[[244   2]
 [  1 245]]
